In [4]:
# 0. Import libraries
import pandas as pd
import numpy as np

# 1. File path (adjust for your machine!)
file_path = "C:/Users/gabri/Downloads/FLATPAY AUG25 STOCK MOVEMENT REPORT.xlsx"


In [5]:
# 2. Load the two production order sheets
prod_dk = pd.read_excel(file_path, sheet_name="Aug25 Prod Order-DK")
prod_de = pd.read_excel(file_path, sheet_name="Aug25 Prod Order - DE")

print("DK Prod Order Columns:", prod_dk.columns.tolist())
print("DE Prod Order Columns:", prod_de.columns.tolist())


DK Prod Order Columns: ['GoodsOwner', 'PackTime', 'ItemNumber', 'Description', 'SerialNumber', 'QuantityPacked', 'ShippingAddress_CustomerName', 'ShippingAddress_Country', 'OrderNumber', 'TrackingNumber']
DE Prod Order Columns: ['GoodsOwner', 'PackTime', 'ItemNumber', 'Description', 'SerialNumber', 'QuantityPacked', 'ShippingAddress_CustomerName', 'ShippingAddress_Country', 'OrderNumber', 'TrackingNumber']


In [6]:
# 3. Function to standardize production orders
def standardize_production(df: pd.DataFrame, source_country_code: str) -> pd.DataFrame:
    rename_map = {
        "OrderNumber": "OrderNumber",
        "ItemNumber": "ItemNumber",
        "Description": "Description",
        "SerialNumber": "SerialNumber",
        "QuantityPacked": "QuantityProduced",
        "PackTime": "ProdTime",
        "ShippingAddress_CustomerName": "CustomerName",
        "ShippingAddress_Country": "DestinationCountry",
        "GoodsOwner": "GoodsOwner",
        "TrackingNumber": "TrackingNumber"
    }
    df = df.rename(columns=rename_map)

    # Ensure all required columns exist
    standard_cols = [
        "OrderNumber","ItemNumber","Description","SerialNumber",
        "QuantityProduced","ProdTime","CustomerName","DestinationCountry",
        "GoodsOwner","TrackingNumber"
    ]
    for c in standard_cols:
        if c not in df.columns:
            df[c] = np.nan

    # Add source (which sheet)
    df["SourceCountry"] = source_country_code

    # Clean datatypes
    df["ProdTime"] = pd.to_datetime(df["ProdTime"], errors="coerce")
    df["QuantityProduced"] = pd.to_numeric(df["QuantityProduced"], errors="coerce")

    # Meta tags
    df["DataType"] = "Flow"
    df["FlowDirection"] = "In"
    df["FlowType"] = "Production"

    ordered = [
        "SourceCountry","OrderNumber","ProdTime","ItemNumber","Description",
        "SerialNumber","QuantityProduced","CustomerName","DestinationCountry",
        "GoodsOwner","TrackingNumber","DataType","FlowDirection","FlowType"
    ]
    return df[ordered]


In [7]:
# 4. Standardize both DK and DE
prod_dk_std = standardize_production(prod_dk.copy(), "DK")
prod_de_std = standardize_production(prod_de.copy(), "DE")

# 5. Combine into one consolidated table
production_all = pd.concat([prod_dk_std, prod_de_std], ignore_index=True)

# 6. Quick preview
production_all.head(10)


,SourceCountry,OrderNumber,ProdTime,ItemNumber,Description,SerialNumber,QuantityProduced,CustomerName,DestinationCountry,GoodsOwner,TrackingNumber,DataType,FlowDirection,FlowType
0,DK,FP-1957,2025-08-01 09:36:00.448919100,Sim-CPOS-2MM#FP,Sim-CPOS-2MM,NaN,32.0,Production Order,DK,Flatpay,3.911827e+09,Flow,In,Production
1,DK,FP-1957,NaT,Cposx5-DE#FP,"Premium Cposx5, Skærm-DE",X1C500064372,1.0,Production Order,DK,Flatpay,3.911827e+09,Flow,In,Production
2,DK,NaN,NaT,Cposx5-DE#FP,"Premium Cposx5, Skærm-DE",X1C500064372,50.0,Production Order,DK,Flatpay,NaN,Flow,In,Production
3,DK,FP-1957,NaT,Cposx5-DE#FP,"Premium Cposx5, Skærm-DE",X1C500064371,1.0,Production Order,DK,Flatpay,3.911827e+09,Flow,In,Production
4,DK,FP-1957,NaT,Cposx5-DE#FP,"Premium Cposx5, Skærm-DE",X1C500063584,1.0,Production Order,DK,Flatpay,3.911827e+09,Flow,In,Production
5,DK,FP-1957,NaT,Cposx5-DE#FP,"Premium Cposx5, Skærm-DE",X1C500063583,1.0,Production Order,DK,Flatpay,3.911827e+09,Flow,In,Production
6,DK,FP-1957,NaT,Cposx5-DE#FP,"Premium Cposx5, Skærm-DE",X1C500064530,1.0,Production Order,DK,Flatpay,3.911827e+09,Flow,In,Production
7,DK,FP-1957,NaT,Cposx5-DE#FP,"Premium Cposx5, Skærm-DE",X1C500064529,1.0,Production Order,DK,Flatpay,3.911827e+09,Flow,In,Production
8,DK,FP-1957,NaT,Cposx5-DE#FP,"Premium Cposx5, Skærm-DE",X1C500064964,1.0,Production Order,DK,Flatpay,3.911827e+09,Flow,In,Production
9,DK,FP-1957,NaT,Cposx5-DE#FP,"Premium Cposx5, Skærm-DE",X1C500064963,1.0,Production Order,DK,Flatpay,3.911827e+09,Flow,In,Production


In [8]:
# 7. Save to files
production_all.to_csv("Production_Consolidated.csv", index=False)
production_all.to_excel("Production_Consolidated.xlsx", index=False)

print("✅ Saved Production_Consolidated.csv and Production_Consolidated.xlsx")


✅ Saved Production_Consolidated.csv and Production_Consolidated.xlsx
